# Import Libaries

In [2]:
pip install flake8

Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip install pyod 

In [4]:
!pip install xgboost

In [5]:
!pip install -U imbalanced-learn
!pip install imblearn
!pip install imblearn
!pip install -c glemaitre imbalanced-learn
!pip install imblearn==0.0

Requirement already up-to-date: imbalanced-learn in c:\users\menna\anaconda3\lib\site-packages (0.8.1)


ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'glemaitre'


In [6]:
!pip install delayed

In [31]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#from sklearn.datasets.samples_generator import make_blobs
from sklearn.pipeline import make_pipeline, make_union,Pipeline
import seaborn as sns
from sklearn.preprocessing import StandardScaler,RobustScaler,LabelEncoder,Normalizer,Binarizer,MinMaxScaler,MaxAbsScaler,PolynomialFeatures
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.utils import shuffle
from sklearn.decomposition import PCA

# Import Data

In [32]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google'

In [33]:
df  = pd.read_csv('creditcard.csv')
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


# Split Data

In [12]:
X = df.iloc[:, df.columns != 'Class']
y = df.iloc[:, df.columns == 'Class']

print(y.value_counts())

Class
0        284315
1           492
dtype: int64


In [13]:
def under_sampling(df):
  number_records_fraud = len (df[df.Class==1])
  fraud_indices = np.array (df[df.Class==1].index)
  normal_indices = np.array (df[df.Class==0].index)
  np.random.seed(42)
  random_normal_indices = np.random.choice (normal_indices, number_records_fraud, replace = False )
  under_sample_indices = np.concatenate ([fraud_indices, random_normal_indices])
  under_sample_data = df.iloc[under_sample_indices,:]
  return under_sample_data

In [14]:
under_sample_data=under_sampling(df)
X_undersample = under_sample_data.iloc [:, under_sample_data.columns != 'Class']
y_undersample = under_sample_data.iloc [:, under_sample_data.columns == 'Class']

print('data after undersampling : ', y_undersample.value_counts())


data after undersampling :  Class
1        492
0        492
dtype: int64


# Train And Test

# GMM
#A Gaussian mixture model is a probabilistic model that assumes all the data points are generated from a mixture of a finite number of Gaussian distributions with unknown parameters. One can think of mixture models as generalizing k-means clustering to incorporate information about the covariance structure of the data as well as the centers of the latent Gaussians.

In [15]:
 from sklearn.mixture import GaussianMixture
gm = GaussianMixture(n_components=2, covariance_type = 'diag', init_params='random', max_iter=100,random_state=0)
gm.fit(X)

GaussianMixture(covariance_type='diag', init_params='random', n_components=2,
                random_state=0)

In [16]:
import sklearn.metrics as metrics
fpr, tpr, thresholds = metrics.roc_curve(y, gm.predict(X))
metrics.auc(fpr, tpr)

0.8808143349534018

### With UnderSampling

In [21]:
under_gm = GaussianMixture(n_components=2, covariance_type = 'diag', init_params='random', max_iter=100,random_state=0)
under_gm.fit(X_undersample)

GaussianMixture(covariance_type='diag', init_params='random', n_components=2,
                random_state=0)

In [22]:
fp, tp, threshold = metrics.roc_curve(y, under_gm.predict(X))
metrics.auc(fp, tp)

0.8931999697175455

# Pyod

#PyOD is a comprehensive and scalable Python toolkit for detecting outlying objects in multivariate data. This exciting yet challenging field is commonly referred as Outlier Detection or Anomaly Detection.

#It is a linear Models for Outlier Detection , type : Probabilistic .

In [23]:
# train the COPOD detector 
from pyod.models.copod import COPOD 
#The whole dataset
clf = COPOD() #COPOD: Copula-Based Outlier Detection
clf.fit(X)

COPOD(contamination=0.1, n_jobs=1)

In [ ]:
COPOD_fpr, COPOD_tpr, COPOD_thresholds = metrics.roc_curve(y, clf.predict(X))
metrics.auc(COPOD_fpr, COPOD_tpr)

### With UnderSampling

In [25]:
under_copod = COPOD()
under_copod.fit(X_undersample)

COPOD(contamination=0.1, n_jobs=1)

In [26]:
COPOD_fpr, COPOD_tpr, COPOD_thresholds = metrics.roc_curve(y, under_copod.predict(X))
metrics.auc(COPOD_fpr, COPOD_tpr)

0.8953353331477497

## Supervised

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1, shuffle=True)  

In [35]:
from sklearn.ensemble import ExtraTreesClassifier
clf = ExtraTreesClassifier(n_estimators=100, random_state=0)
clf.fit(X_train, y_train)
fp, tp, threshold = metrics.roc_curve(y_test, clf.predict(X_test))
metrics.auc(fp, tp)

<ipython-input-35-26db46bf915c>:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(X_train, y_train)


0.8851084937600139

### With UnderSampling

In [36]:
clf = ExtraTreesClassifier(n_estimators=100, random_state=0)
clf.fit(X_undersample, y_undersample)
fp, tp, threshold = metrics.roc_curve(y, clf.predict(X))
metrics.auc(fp, tp)

<ipython-input-36-e9727ead0181>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(X_undersample, y_undersample)


0.991483038179484

# summary :

Under sampling  is useful with the  case of Supervised learning .
